In [1]:
import PyPDF2
import os
import re
import string
import random

PARTIJPATH = 'partijprogrammas'
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")


In [2]:
def basic_tokenizer(sentence):
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
    return [w for w in words if w]

def get_index_end_sentence(tokens):
    for index,token in enumerate(tokens):
        if token in ["?","!","."]:
            return index
    return len(tokens)
def get_sentences_from_tokens(tokens):
    sentences = []
    while len(tokens) > 0:
        nextindex = get_index_end_sentence(tokens)
        if nextindex == len(tokens):
            sentences.append(tokens)
            return sentences
        else:
            sentences.append(tokens[:nextindex+1])
            tokens = tokens[nextindex+1:]
    return sentences
def get_parties_and_sentences():
    partijprogrammas = os.listdir(PARTIJPATH)
    part_sentences = dict()
    
    
    for partijprogramma_name in partijprogrammas:
        print(partijprogramma_name)
        pdf_obj = open(os.path.join(PARTIJPATH,partijprogramma_name),'rb')
        pdfreader = PyPDF2.PdfFileReader(pdf_obj)

        sentences = []
        
        for page in pdfreader.pages:
            text = page.extractText()
            # remove all words with a number
            text = text.replace("\n"," ")

            text = re.sub(r'\w*\d\w*', '', text).strip()
            
            # make lower case
            text = text.lower()
            if(len(text)>0):
                tokens = basic_tokenizer(text)
                sentences_this_page = get_sentences_from_tokens(tokens)
                sentences.extend(sentences_this_page)
        part_sentences[partijprogramma_name] = sentences
    return part_sentences

parties_and_sentences = get_parties_and_sentences()
print("done!")


PPNL.pdf
CDA.pdf
LidK.pdf
PartijVoorDeDieren.pdf


VVD.pdf
DBB.pdf
SP.pdf
OndernemersPartij.pdf
VrijzinnigePartij.pdf
VNL.pdf
ChristenUnie.pdf
AR1.pdf
50PLUS.pdf
NieuweWegen.pdf
PvvNew.pdf
done!


In [3]:
print([a for a in parties_and_sentences.keys()])
print(parties_and_sentences['SP.pdf'][100])
vocab = dict()
_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_UNK = "_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]


for party in parties_and_sentences.keys():
    for line in parties_and_sentences[party]:
        for word in line:
            if not word in vocab:
                vocab[word]=0
            vocab[word] +=1
        
sorted_vocab = sorted(vocab, key=vocab.get, reverse=True)
sorted_vocab = [word for word in sorted_vocab if vocab[word] >= 5]
vocab_list = _START_VOCAB + sorted_vocab
print(len(vocab_list))
for uncommon in vocab_list[-20:]:
    print(uncommon + " : " + str(vocab[uncommon]))
    
class IdOfWordGetter:
    def get_word_of_id(self,index):
        return self.vocab[index]
    def get_id_of_word(self,word):
        if word in self.word_dict:
            return self.word_dict[word]
        else:
            return self.word_dict[_UNK]
    def __init__(self,vocab):
        self.vocab = vocab
        self.word_dict = dict()
        for index,word in enumerate(vocab):
            self.word_dict[word] = index
id_of_word_getter = IdOfWordGetter(vocab_list)
party_count = len(parties_and_sentences.keys())
#if len(vocab_list) > max_vocabulary_size:
#     vocab_list = vocab_list[:max_vocabulary_size]
# with gfile.GFile(vocabulary_path, mode="wb") as vocab_file:
# for w in vocab_list:
#   vocab_file.write(w + b"\n")




['PvvNew.pdf', 'OndernemersPartij.pdf', 'CDA.pdf', '50PLUS.pdf', 'VNL.pdf', 'VrijzinnigePartij.pdf', 'VVD.pdf', 'DBB.pdf', 'NieuweWegen.pdf', 'PartijVoorDeDieren.pdf', 'ChristenUnie.pdf', 'LidK.pdf', 'AR1.pdf', 'SP.pdf', 'PPNL.pdf']
['ministeries', 'richten', 'hun', 'verantwoording', 'beter', 'in', ',', 'zodat', 'altijd', 'duidelijk', 'is', 'of', 'het', 'geld', 'goed', 'wordt', 'besteed', '.']
4679
misbruikt : 5
geschoold : 5
fan : 5
gesjoemel : 5
visumvrij : 5
huisdier : 5
landmacht : 5
a˙ : 5
telegraaf : 5
klant : 5
bondgenootschap : 5
amri : 5
netto : 5
misdrijf : 5
overvolle : 5
bomen : 5
very : 5
beperkte : 5
bondgenootschappelijk : 5
windparken : 5


In [4]:
import random
def load_data(parties_and_sentences,sorted_vocab,getter):
    labels = parties_and_sentences.keys()
    train_x = list()
    train_y = list()
    for party_index,name in enumerate(parties_and_sentences.keys()):
        print(name + " has "  + str(len(parties_and_sentences[name])) + " sentences ")
        for sentence in parties_and_sentences[name]:
            numeric_sentence = list()
            for word in sentence:
                numeric_sentence.append(getter.get_id_of_word(word))
            train_x.append(numeric_sentence)
            train_y.append(party_index)
    train_set = list(zip(train_x,train_y))
    random.shuffle(train_set)
    percentage_split = 0.8
    train = train_set[:int(percentage_split*len(train_set))]
    test = train_set[int(percentage_split*len(train_set)):]
    return zip(*train),zip(*test)


(X_train, y_train), (X_test, y_test) = load_data(parties_and_sentences,sorted_vocab,id_of_word_getter)
    

PvvNew.pdf has 208 sentences 
OndernemersPartij.pdf has 318 sentences 
CDA.pdf has 1435 sentences 
50PLUS.pdf has 596 sentences 
VNL.pdf has 378 sentences 
VrijzinnigePartij.pdf has 6236 sentences 
VVD.pdf has 2808 sentences 
DBB.pdf has 433 sentences 
NieuweWegen.pdf has 1023 sentences 
PartijVoorDeDieren.pdf has 1511 sentences 
ChristenUnie.pdf has 2631 sentences 
LidK.pdf has 489 sentences 
AR1.pdf has 396 sentences 
SP.pdf has 902 sentences 
PPNL.pdf has 48 sentences 


In [5]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils





import numpy as np
# fix random seed for reproducibility
np.random.seed(7)
top_words = len(vocab_list)
#top_words = 5000
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

Using TensorFlow backend.


In [6]:
print(y_train[3])

6


In [7]:


# truncate and pad input sequences
max_review_length = 20
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
y_train = np_utils.to_categorical(y_train, party_count)
prevytest = y_test
y_test = np_utils.to_categorical(y_test, party_count)


In [8]:
print(X_train[3])

[ 538    5 1717  833   19 2779  961  253    6  538    5 1083 4544 4168   12
   42 1782  285  170    4]


In [9]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(party_count, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
print(model.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 20, 32)        149728      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 15)            1515        lstm_1[0][0]                     
Total params: 204,443
Trainable params: 204,443
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [10]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64,verbose=1)

Train on 15529 samples, validate on 3883 samples
Epoch 1/3
 2752/15529 [====>.........................] - ETA: 14s - loss: 2.2229 - categorical_accuracy: 0.3289 

KeyboardInterrupt: 

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
classes_predicted = model.predict_classes(X_test, batch_size=32, verbose=1)
for i in range(100):
    for id in X_test[i]:
        print(id_of_word_getter.get_word_of_id(id),end=' ')
    print()
    print("Predicted: " + list(parties_and_sentences.keys())[classes_predicted[i]])
    print("Actual: " + list(parties_and_sentences.keys())[prevytest[i]])


In [1]:
import requests

url = "http://verkiezingsradar.bigdatarepublic.nl/fit"

payload = "{\"text\":\"buitenlanders muur duitsland \"}"
headers = {
    'origin': "http://verkiezingsradar.bigdatarepublic.nl",
    'x-devtools-emulate-network-conditions-client-id': "b55ebfed-6d07-4e1c-a42e-be325f4cf5ae",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.74 Safari/537.36",
    'content-type': "application/json",
    'accept': "*/*",
    'referer': "http://verkiezingsradar.bigdatarepublic.nl/",
    'accept-encoding': "gzip, deflate",
    'accept-language': "en-US,en;q=0.8",
    'cache-control': "no-cache",
    'postman-token': "30436b82-4674-804e-f23d-449f3c084097"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

[["PVDA", 0.044005141784259585], ["SGP", 0.05528280809980826], ["Ondernemerspartij", 0.04940984983017636], ["Libertarische Partij", 0.047463126249664755], ["Lokaal in de Kamer", 0.02826472527306533], ["Groenlinks", 0.04497192106767064], ["PVV", 0.04104062872838321], ["VoorNederland", 0.03542658908898007], ["Piratenpartij", 0.04513484025004688], ["DENK", 0.03610091314620288], ["De Burger Beweging", 0.03409747263676953], ["D66", 0.048552280663354964], ["CDA", 0.04168068154635131], ["50PLUS", 0.04334079740266899], ["VVD", 0.04301567534435939], ["Partij voor de Dieren", 0.04474988697673197], ["MenS en Spirit/Basisinkomen Partij/V", 0.04444580872763036], ["ChristenUnie", 0.05130411086421898], ["SP", 0.03944912480936048], ["NIEUWE WEGEN", 0.04892838231563123], ["JEZUS LEEFT", 0.03813739435665307], ["Vrijzinnige Partij", 0.05012862404482335], ["Forum voor Democratie", 0.04506921679318823]]


In [2]:
import json

array = '{"fruits": ["apple", "banana", "orange"]}'
data  = json.loads(response.text)

In [3]:
print(data)

[['PVDA', 0.044005141784259585], ['SGP', 0.05528280809980826], ['Ondernemerspartij', 0.04940984983017636], ['Libertarische Partij', 0.047463126249664755], ['Lokaal in de Kamer', 0.02826472527306533], ['Groenlinks', 0.04497192106767064], ['PVV', 0.04104062872838321], ['VoorNederland', 0.03542658908898007], ['Piratenpartij', 0.04513484025004688], ['DENK', 0.03610091314620288], ['De Burger Beweging', 0.03409747263676953], ['D66', 0.048552280663354964], ['CDA', 0.04168068154635131], ['50PLUS', 0.04334079740266899], ['VVD', 0.04301567534435939], ['Partij voor de Dieren', 0.04474988697673197], ['MenS en Spirit/Basisinkomen Partij/V', 0.04444580872763036], ['ChristenUnie', 0.05130411086421898], ['SP', 0.03944912480936048], ['NIEUWE WEGEN', 0.04892838231563123], ['JEZUS LEEFT', 0.03813739435665307], ['Vrijzinnige Partij', 0.05012862404482335], ['Forum voor Democratie', 0.04506921679318823]]


In [ ]:
for partyandscore in data:
    print(partyandscore[0])

PVDA
SGP
Ondernemerspartij
Libertarische Partij
Lokaal in de Kamer
Groenlinks
PVV
VoorNederland
Piratenpartij
DENK
De Burger Beweging
D66
CDA
50PLUS
VVD
Partij voor de Dieren
MenS en Spirit/Basisinkomen Partij/V
ChristenUnie
SP
NIEUWE WEGEN
JEZUS LEEFT
Vrijzinnige Partij
Forum voor Democratie
